In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

cat, pgram = pickle.load(open("out/results.pkl", "rb"))
periods = pgram[0]
sigls = pgram[1][0]
sigce = pgram[1][1]
sigaov = pgram[1][2]

In [17]:
def fold_for_axis(t, y, p):
    x = t % p
    x = x / p
    x = np.concatenate([x, x + 1])
    y = np.concatenate([y, y])
    return x,y

def plot_folds(row):
    title = row["designation"] + "ref"
    t = row["time"]
    y = row["mag"]
    p_t = row["period"]


    p1, ps1 = row["peak1"], row["peak1_sig"]
    p2, ps2 = row["peak2"], row["peak2_sig"]
    plp, pslp = row["best_long_period"], row["best_long_period_sig"]

    # make 4-part subplots
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))

    # top right: full light curve
    axs[0, 0].plot(t, y, ".", label="Full Light Curve")
    axs[0, 1].plot(*fold_for_axis(t, y, p_t), ".", label="Correct Period: {}".format(p_t))
    axs[1, 0].plot(*fold_for_axis(t, y, p1), ".", label=f"Peak 1: {p1} @ sig {ps1}")
    axs[1, 1].plot(*fold_for_axis(t, y, p2), ".", label=f"Peak 2: {p2} @ sig {ps2}")

    # title each subplot
    axs[0, 0].set_title("Full Light Curve")
    axs[0, 1].set_title("Correct Period: %.5f" % p_t)
    axs[1, 0].set_title(f"Peak 1: {p1} @ sig {np.round(ps1, 3)}")
    axs[1, 1].set_title(f"Peak 2: {p2} @ sig {np.round(ps2, 3)}")

    return fig

def plot_pgram(periods, sigls, sigce, sigaov):
    x = periods 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=np.concatenate((np.zeros(len(x) - len(sigls)), sigls)), mode='lines', name='Lomb-Scargle'))
    fig.add_trace(go.Scatter(x=x, y=sigaov, mode='lines', name='AOV'))

    # make x log-scale
    fig.update_xaxes(type="log")

    return fig

i = 0

In [ ]:
NAME = "VarWISE J054556.98-703604.2"
row = cat.loc[cat["designation"] == NAME]
i = row.index[0]
row = row.iloc[0]
print(row)

print("Shortcadence: ", np.median(np.diff(row["time"])))
folds = plot_folds(row)
folds.show()
pgrams = plot_pgram(periods, sigls[i], sigce[i], sigaov[i])
pgrams.show()
i += 1